In [1]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use

%matplotlib qt5

# To switch from one to another, you have to select Kernel->Restart

# %matplotlib inline

from modsim import *
from numpy import *
import numpy as np

In [2]:
"""flu"""
tc_flu = 2      # time between contacts in days 
tr_flu = 10      # recovery time in days

beta_flu = 1 / tc_flu      # contact rate in per day
gamma_flu = 1 / tr_flu     # recovery rate in per day
days = 2

In [3]:
def make_system(beta, gamma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(S=99, I=1, R=0)
    init /= np.sum(init)
    
    
    t0 = 0
    t_end = 7*32 # 224days 

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma)


In [4]:
def update1(state, system):
    """Update the SIR model.
    
    state: State (s, i, r)
    system: System object
    
    returns: State (sir)
    """
    unpack(system)
    s, i, r = state

    infected = beta * i * s
    recovered = gamma * i
    
    s -= infected 
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)

In [5]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
    
    Add a TimeFrame to the System: results
    
    system: System object
    update_func: function that updates state
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.loc[t0] = init
    
    for i in linrange(t0, t_end):
        frame.loc[i+1] = update_func(frame.loc[i], system)
    
    system.results = frame

In [6]:
def add_quarantine(system, fraction):
    high = 2
    tc_flu = fraction * high + high 
    system.beta = 1 /tc_flu

In [7]:
def add_medicine(system, days):
    system.gamma = 1/ (tr_flu - days)

In [8]:
flu_q = make_system(beta_flu, gamma_flu)
add_quarantine(flu_q, 0.3)
run_simulation(flu_q, update1)

In [9]:
def plot_results(S, I, R, title):
    plot(S, '--', color='blue', label = 'Susceptible')
    plot(I, '-', color='red', label = 'Infected')
    plot(R, ':', color='green', label = 'Resistant')
    decorate(xlabel='Time (days)',
         ylabel='Fraction of population', title=title)

In [10]:
newfig()
plot_results(flu_q.results.S, flu_q.results.I, flu_q.results.R, title = 'Flu SIR Model With Qurantine')

In [11]:
flu_m = make_system(beta_flu, gamma_flu)
add_medicine(flu_m, 2)
run_simulation(flu_m, update1)

In [12]:
newfig()
plot_results(flu_m.results.S, flu_m.results.I, flu_m.results.R, title="Flu SIR Model With Medicine")

In [13]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
    
    Add three Series objects to the System: S, I, R
    
    system: System object
    update_func: function that updates state
    """
    S = TimeSeries()
    I = TimeSeries()
    R = TimeSeries()

    state = system.init
    t0 = system.t0
    S[t0], I[t0], R[t0] = state
    
    for i in linrange(system.t0, system.t_end):
        state = update_func(state, system)
        S[i+1], I[i+1], R[i+1] = state
    
    system.S = S
    system.I = I
    system.R = R

In [14]:
tc = 2      # time between contacts in days 
tr = 10      # recovery time in days

beta = 1 / tc      # contact rate in per day
gamma = 1 / tr     # recovery rate in per day

system = make_system(beta, gamma)
run_simulation(system, update1)

In [15]:
def plot_results(S, I, R):
    """Plot the results of a SIR model.
    
    S: TimeSeries
    I: TimeSeries
    R: TimeSeries
    """
    plot(S, '--', color='blue', label='Susceptible')
    plot(I, '-', color='red', label='Infected')
    plot(R, ':', color='green', label='Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of population',
             title = "Pox SIR model without any treatment")

In [16]:
plot_results(system.S, system.I, system.R)

In [17]:
def plot_comparison(S_m, S_q, title):
    plot(S_m, '-', color='blue', label = 'Medicine')
    plot(S_q, '-', color='red', label = 'Quarantine')
    plot(system.S, '-', color='green', label = 'No treatment')
    decorate(xlabel='Time (days)',
         ylabel='Fraction of population', title=title)

In [18]:
newfig()
plot_comparison(flu_m.results.S, flu_q.results.S, title = "Susceptible Population Over Time")

In [19]:
newfig()
plot_comparison(flu_m.results.R, flu_q.results.R, title = "Resistant Population Over Time")